In [4]:
import json
import urllib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import plot_map
import matplotlib as mpl
import seaborn as sns
import sys
import os
import basics
import geopandas as gpd
import pymysql
from shapely.geometry import Point 
from shapely.geometry import Polygon
from shapely.geometry import LineString
import pyproj
from urllib.parse import quote
import string
import math
from coord_convert.transform import wgs2gcj, wgs2bd, gcj2wgs, gcj2bd, bd2wgs, bd2gcj 
import importlib
importlib.reload(basics)

<module 'basics' from 'C:\\Users\\Administrator\\Desktop\\poi爬取\\basics.py'>

# 1.获取荆州市各类型POI

In [5]:
# 关键字搜索，每次最多返回1000个POI信息，适合数量较少的POI类型
def getPOIKeywords(poitype,citycode):
    POIList = []
    for page in range(1 , 46):
        url = "https://restapi.amap.com/v3/place/text?key="+ak+"&keywords=&types="+poitype+"&city="+citycode+\
        "&children=l&offset=20&page="+str(page)+"&extensions=all"
        json_obj = urllib.request.urlopen(url)
        json_data = json.load(json_obj)
        try:
            pois = json_data['pois']
        except Exception as e:
            print("错误",url)
            print(e)
            continue
        if (pois != []):
            for j in range(0,len(pois)):
                poi_j = pois[j]
                id = poi_j['id']
                lon = float(poi_j['location'].split(',')[0])
                lat = float(str(poi_j['location']).split(',')[1])
                name = poi_j['name']
                poi = basics.PointWithAttr(id,lon,lat,poitype,name)
                POIList.append(poi)
    return POIList

In [6]:
#范围搜索，将行政区划分为num个子区域，用以无遗漏采集POI
def getPOIPolygon(poitype,citycode,num):
    POIList = []
    # 获取citycode对应的行政边界
    districtBoundryUrl = "http://restapi.amap.com/v3/config/district?key="+ak+"&keywords="+citycode+"&subdistrict=0&extensions=all"
    json_obj=urllib.request.urlopen(districtBoundryUrl)
    json_data=json.load(json_obj)
    districts = json_data['districts']
    if len(districts) == 0:
        # 0意味着该区域无法搜索出行政区边界信息
        print("该区域无法搜索出行政区边界信息")
        return []
    polyline = districts[0]['polyline']
    center = districts[0]['center']
    pointscoords = polyline.split(';')
    newlinelength=int(len(pointscoords) / num)  # 每个子区域的点数目
    # 将行政区域划分为num个子区域，获得每个子区域的poi
    boundryMarks = [] # 标记行政区域的划分点
    for i in range(0,len(pointscoords) , newlinelength):
        boundryMarks.append(i)
    boundryMarks.append(len(pointscoords) - 1)
    for i in range(0 , len(boundryMarks) - 1):
        firstMark = boundryMarks[i]
        lastMark = boundryMarks[i+1]
        newboundry = [center] #新的多边形划分子区，从中心开始
        for j in range(firstMark , lastMark+1):
            newboundry.append(pointscoords[j])
        newboundry.append(center)  #最后回到center，达到闭合
        # newboundryStr是多边形的边界
        newboundryStr = "|".join(newboundry)
        for page in range(1,46):
            url="http://restapi.amap.com/v3/place/polygon?key="+ak+"&polygon="+newboundryStr+"&keywords=&types="+poitype+ \
            "&offset=20&page="+str(page)+"&extensions=all"
            try:
                json_obj=urllib.request.urlopen(url)
            except Exception as e:
                print("错误" , url)
                print(e)
                continue
            json_data=json.load(json_obj)
            try:
                pois=json_data['pois']
            except Exception as e:
                print("错误" , url)
                print(e)
                continue
            if (pois != []):
                for j in range(0,len(pois)):
                    poi_j=pois[j]
                    id=poi_j['id']
                    lon=float(poi_j['location'].split(',')[0])
                    lat=float(str(poi_j['location']).split(',')[1])
                    name=poi_j['name']
                    poi=basics.PointWithAttr(id,lon,lat,poitype,name)
                    POIList.append(poi)
    return POIList

In [7]:
# 将poi写入txt文件ok
def writePOIs2File(POIList,outputfile):
    f = open(outputfile,'a' ,encoding="utf-8")
    for i in range(0 , len(POIList)):
        f.write(POIList[i].id+";"+POIList[i].name+";"+POIList[i].type+';'+str(POIList[i].lon)+';'+str(POIList[i].lat)+"\n")
    f.close()    

In [1]:
# 密钥ak
ak= ''
# 将采集的POI数据保存在output_directory文件夹里
output_directory="C:\\Users\\Administrator\\Desktop\\poi爬取\\"
if not os.path.exists(output_directory+"poi\\"):
    os.mkdir(output_directory+"poi\\")
try:
    typefile=open(output_directory+"poitype.txt",'r')
except Exception as e:
    print('错误提示:请将poitype.txt复制到'+output_directory+"下")
poitypeList=typefile.readlines()
for poitype in poitypeList:  # 遍历各种类型，并下载每种类型的poi
    poitype=poitype.split('\n')[0]  # 去掉换行符
    #行政区域编码citycode
    citycodes={'沙市区':'421002','荆州区':'421003','公安县':'421022','监利市':'421023','江陵县':'421024','石首市':'421081',\
               '洪湖市':'421083','松滋市':'421087'}
#     citycodes={'杭州市':'330100'}
    for citycode in citycodes.values():  # 遍历行政区域编码，将各个行政区域的poi都下载下来
        #调用关键字搜索的getPOIKeywords函数，获取POI数量较少的案例
        try:
            POIList = getPOIKeywords(poitype , citycode)
        except Exception as e:
            print("错误提示:" , e)
            POIList = range(1500)
        # 关键字搜索返回POI数量大于900个，则调用多边形搜索
        # getPOIPolygon函数，获取POI数量较多的类别
        if (len(POIList)>=900):
            POIList = []
            POIList=getPOIPolygon(poitype,citycode , 30)
            outputfile1=output_directory+"poi\\"+poitype+"_"+citycode+"_polygon.txt"
            f=open(outputfile1,'w',encoding="utf-8")
            f.close()
            print(citycode , poitype,len(POIList))
            writePOIs2File(POIList,outputfile1)
        else:
            outputfile = output_directory+"poi\\"+poitype+"_"+citycode+"_keywords.txt"
            f=open(outputfile,'w',encoding="utf-8")
            f.close()
            print(citycode,poitype,len(POIList))
            writePOIs2File(POIList,outputfile)
# 合并poitype类型的文本文件
basics.mergetxt(output_directory+"poi\\",output_directory+"final.txt")

NameError: name 'os' is not defined